In [69]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

There is accurate data for `df['College_chosen_by_non-matrics']` for the last three years only.

In [70]:
df = pd.read_csv("../data/raw/CriticalPath_Data_EM_Confidential.csv")
_2017 = pd.read_csv('../data/raw/201730_Fresh_Not_Enrolled_College_Choice.csv').set_index("SARADAP_PIDM")
_2018 = pd.read_csv('../data/raw/201830_Fresh_Not_Enrolled_College_Choice.csv').set_index("SARADAP_PIDM")
_2019 = pd.read_csv('../data/raw/201930_Fresh_Not_Enrolled_College_Choice.csv').rename(columns={"PIDM":"SARADAP_PIDM"}).set_index("SARADAP_PIDM")

last_three_years_df = pd.concat([_2017,_2018,_2019])
mapper = last_three_years_df.to_dict()

df = df.drop(columns='College_chosen_by_non-matrics')
df['College_chosen_by_non-matrics'] = df['Unique_student_ID'].map(mapper['CollegeName'])

In [71]:
df['Enrolled'] = df['Enrolled'].map({80:True})
df['Enrolled'] = df['Enrolled'].fillna(False)

df = df[(df['Application_Type']!='AM') & (df['Application_Type']!='HE')]

Get info for `object` types

In [72]:
# df.select_dtypes('object').info()

Info for `int` and `floats`

In [73]:
# df.select_dtypes([int,float]).info()

## Columns to dump as said by Ned:
* tuition_waiver...
* outside_aid - not aquired till late in the process
* athletic_based_inst_aid
* internal_academic_rating
* deposit date

In [74]:
df = df.drop(columns=['Tuition_waivers_and_exchanges','Outside_aid','Athletic_based_inst_aid','Internal_Academic_rating'])

### Some columns are y/n, but the n values are `NaN's`. Fix this.

* 'Ints' columns
* Test_optional
* Legacy $\to$ there are many codes, all of which mean that in some way they have family that have attended here before.

In [75]:
for col in df.columns.values:
    if col.startswith("Ints"):
        df[col] = df[col].map(dict(zip(df[col].unique(), [0,1])))

In [76]:
# for i,col in enumerate(df.columns.values):
#     print(col,df.count().values[i])

### Dump all columns with less than 3,000 values.  These are all noise/for enrolled students ONLY

In [77]:
df = df[df.columns.values[df.count()>3000]]

### One hot-encode commuter/residence students.

In [78]:
df['COMM'] = pd.get_dummies(df['Dorm_or_commuter_student'])['COMM']
df['RESD'] = pd.get_dummies(df['Dorm_or_commuter_student'])['RESD']

### One hot-encode School of Science/Business/Arts

In [79]:
df[['AD','BD','SD']] = pd.get_dummies(df['CollegeCode'])

### One hot-encode HD_Academic_Rating

In [80]:
df[['AR1','AR1B','AR2','AR3','AR4','AR5']] = pd.get_dummies(df['HD_Academic_Rating']).drop(columns=['7','722','ARX'])

### Make `Test_Optional` a bool type

In [86]:
df['Test_Optional'] = df['Test_Optional'].map({'TOPT':True})
df['Test_Optional'] = df['Test_Optional'].fillna(False)

### Drop One hot-encoded columns (originals)

In [89]:
df = df.drop(columns=['Dorm_or_commuter_student','CollegeCode','HD_Academic_Rating'])

### These columns repeat information we know

In [100]:
df =  df.drop(columns=['Admission_status','HS_Numeric_rank'])

### Make legacy students a T/F

In [109]:
df['Legacy'] = ~df['Legacy'].isnull().astype(int) + 2  # any legacy student is assigned a 1, otherwise a 0

### One hot-encode Ethnicity

In [113]:
df[df['Ethnicity'].unique()] = pd.get_dummies(df['Ethnicity'])

In [116]:
df = df.drop(columns='Unknown').rename(columns={"Multiple":"Multiple_races"})

In [149]:
# df.select_dtypes(exclude='object').info()

### Write to file

In [147]:
df.to_csv('../data/interim/Third_order_clean_confidential.csv')